In [ ]:
pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np  
import math
mp_drawing = mp.solutions.drawing_utils  # get drawing utilities from MediaPipe
mp_pose = mp.solutions.pose  # grabbing the Pose model from MediaPipe

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
#cap = cv2.VideoCapture("http://10.0.0.185:8080/video")
cap = cv2.VideoCapture(1)
running = False
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.6) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        k = cv2.waitKey(33) # THIS GRABS THE KEY VALUE
        print(k)

        # Show the wireframe
        if(k == 116):
            running = True
        # Turn off the wireframe.
        if(k == 121):
            running = False
        
        
        if(running):
            try:

                

                landmarks = results.pose_landmarks.landmark

                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                r_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

                l_alt_angle = calculate_angle(right_shoulder, left_shoulder, left_elbow)
                r_alt_angle = calculate_angle(left_shoulder, right_shoulder, right_elbow)
                
                up_wrong = abs(545-464)/545
                down_wrong = abs(630-464)/545

                form = "Bad form"
                cv2.putText(image, str(l_alt_angle), (100,100), cv2.FONT_HERSHEY_TRIPLEX, 2.5, (0,0,0), 2)
                # cv2.putText(image, str(left_elbow[1] * cap.get(3)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (255,255,255), 2)
                
                # if(l_alt_angle < 165):
                #     form = "Bad form"
                #     cv2.putText(image, "Left side: " + str(form), (200,500), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), 2)

                # elif(l_alt_angle > 165 and l_alt_angle < 183):
                #     form = "Good Form"
                #     cv2.putText(image, "Left side: " + str(form), (200,800), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,255,0), 2)
                
                # if(r_alt_angle < 165):
                #     form = "Bad form"
                #     cv2.putText(image, "Right side: " + str(form), (200,300), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), 2)

                # elif(r_alt_angle > 165 and l_alt_angle < 183):
                #     form = "Good Form"
                #     cv2.putText(image, "Right side: " + str(form), (200,300), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

                if ((left_elbow[1]*cap.get(3)) < (cap.get(3) * (left_shoulder[1] - left_shoulder[1] * up_wrong))):
                    form = "Bad form"
                    cv2.putText(image, "TEST: " + str(form), (200,500), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)

                if ((left_elbow[1]*cap.get(3)) > (cap.get(3) * (left_shoulder[1] + left_shoulder[1] * up_wrong))):
                    form = "Bad form"
                    cv2.putText(image, "TEST: " + str(form), (400,500), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)

                if l_angle < 90:
                    form = "Bad form"
                    cv2.putText(image, "Left Side: " + str(form), (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)

                elif l_angle >= 90:
                    form = "Good Form"
                    cv2.putText(image, "Left Side: " + str(form), (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

                if r_angle < 90:
                    form = "Bad form"
                    cv2.putText(image, "Right Side: " + str(form), (100,300), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)

                elif r_angle >= 90:
                    form = "Good Form"
                    cv2.putText(image, "Right Side: " + str(form), (100,300), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
                
                if left_elbow[1] < (left_shoulder[1] + (left_shoulder[1]*.05)):
                    #form = "Bad elbow height"
                    cv2.putText(image,"Bad elbow height", (100,400), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)

            except:
                pass

            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(145,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(100,66,230), thickness=2, circle_radius=2) 
                                    )               
            
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    results.release()
    cap.release()
    
    cv2.destroyAllWindows()

In [ ]:
left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

In [ ]:
## EXERCISE: BICEP CURL


cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("http://10.0.0.185:8080/video")
with mp_pose.Pose(static_image_mode = False, smooth_landmarks=True,enable_segmentation = False, model_complexity = 1, min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:

            

            landmarks = results.pose_landmarks.landmark

            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            r_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            l_alt_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
            r_alt_angle = calculate_angle(right_elbow, right_shoulder, right_hip)

            form = "Bad form"
            
            cv2.putText(image, "Bicep Curls", (500,50), cv2.FONT_HERSHEY_TRIPLEX, 2, (255,200,200), 2)

            
            if(l_alt_angle > 30):
                form = "Bad form"
                cv2.putText(image, "Bad form. Keep your left elbow in.: " + str(form), (200,500), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), 2)

            if(r_alt_angle > 30):
                form = "Bad form"
                cv2.putText(image, "Bad form. Keep your right elbow in.: " + str(form), (200,500), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), 2)

            

        except:
            pass

        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(145,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(100,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    results.release()
    cv2.destroyAllWindows()

##### left_elbow, left_shoulder, left_wrist, right_elbow, right_shoulder, right_wrist

In [ ]:
calculate_angle(left_shoulder, left_elbow, left_wrist), calculate_angle(right_shoulder, right_elbow, right_wrist)

In [ ]:
#Y pose
#cap = cv2.VideoCapture("http://10.0.0.185:8080/video")
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("Y_pose.mp4")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_path = "Y_pose_test.mp4"
fps_out = 25 # frame rate (frames per second), over 30 may not work
out = cv2.VideoWriter(video_path, fourcc, fps_out, (width, height))

running = False
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.6) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        k = cv2.waitKey(33) # THIS GRABS THE KEY VALUE

        # Show the wireframe
        if(k == 116):
            running = True
        # Turn off the wireframe.
        if(k == 121):
            running = False
        
        
        if(running):
            
            
            try:
                landmarks = results.pose_landmarks.landmark
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                ls_visibility = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                rs_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
                
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                le_visibility = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                re_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
                
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                rw_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                lw_visibility = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
                
                left_knee_angle = calculate_angle(left_ankle, left_knee, left_hip)
                right_knee_angle = calculate_angle(right_ankle, right_knee, right_hip)
                
                right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
                left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
                
                left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                
                l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                r_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                
                l_alt_angle = calculate_angle(right_shoulder, left_shoulder, left_elbow)
                r_alt_angle = calculate_angle(left_shoulder, right_shoulder, right_elbow)
                
                left_arm_visible = ls_visibility>0.8 and le_visibility>0.8 and lw_visibility>0.8
                right_arm_visible = rs_visibility>0.8 and re_visibility>0.8 and rw_visibility>0.8
                
                
                
                
                if left_knee_angle>170 and right_knee_angle>170 and left_hip_angle>150 and right_hip_angle>150:
                    isCorrect = "True"
                    color = (0,255,0)
                else:
                    isCorrect = "False"
                    color = (0,0,255)
                cv2.putText(image,"Lower body position Okay: "+isCorrect, (25,25), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                
                if (left_arm_visible):
                    
                    #checking if left arm is straight
                    if l_angle>150:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    cv2.putText(image,"Left arm okay: " +isCorrect, (25,50), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                    #Checking if the angle between shoulder and left arm is correct
                    if l_alt_angle>110 and l_alt_angle<150:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    
                    cv2.putText(image,"Left shoulder okay: "+isCorrect, (25,75), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                else:
                    cv2.putText(image,"Left arm is not visible properly ", (25,50), cv2.FONT_HERSHEY_TRIPLEX, 1, (250, 248, 3), 1)
                
                
                if (right_arm_visible):
                    
                    #checking if right arm is straight
                    if r_angle>150:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    cv2.putText(image,"Right arm okay: "+isCorrect, (25,100), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                    #Checking if the angle between shoulder and left arm is correct
                    if r_alt_angle>110 and r_alt_angle<150:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    cv2.putText(image,"right shoulder okay: "+isCorrect, (25,125), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                else:
                    cv2.putText(image,"Right arm is not visible properly ", (25,100), cv2.FONT_HERSHEY_TRIPLEX, 1, (252, 248, 3), 1)


            except:
                pass

            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(145,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(100,66,230), thickness=2, circle_radius=2) 
                                    )                            
        out.write(image)   
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    out.release()
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#W pose
#cap = cv2.VideoCapture("http://10.0.0.185:8080/video")
cap = cv2.VideoCapture("W_pose.mp4")


width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_path = 'W_pose_test.mp4'
fps_out = 25 # frame rate (frames per second), over 30 may not work
out = cv2.VideoWriter(video_path, fourcc, fps_out, (width, height))

running = False
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.6) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        k = cv2.waitKey(33) # THIS GRABS THE KEY VALUE

        # Show the wireframe
        if(k == 116):
            running = True
        # Turn off the wireframe.
        if(k == 121):
            running = False
        
        
        if(running):
            
            
            try:
                landmarks = results.pose_landmarks.landmark
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                ls_visibility = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                rs_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
                
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                le_visibility = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                re_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
                
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                rw_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                lw_visibility = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
                
                left_knee_angle = calculate_angle(left_ankle, left_knee, left_hip)
                right_knee_angle = calculate_angle(right_ankle, right_knee, right_hip)
                
                right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
                left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
                
                left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                
                l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                r_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                
                l_alt_angle = calculate_angle(right_shoulder, left_shoulder, left_elbow)
                r_alt_angle = calculate_angle(left_shoulder, right_shoulder, right_elbow)
                
                left_arm_visible = ls_visibility>0.8 and le_visibility>0.8 and lw_visibility>0.8
                right_arm_visible = rs_visibility>0.8 and re_visibility>0.8 and rw_visibility>0.8
                
                
                
                if left_knee_angle>170 and right_knee_angle>170 and left_hip_angle>150 and right_hip_angle>150:
                    isCorrect = "True"
                    color = (0,255,0)
                else:
                    isCorrect = "False"
                    color = (0,0,255)
                cv2.putText(image,"Lower body position Okay: "+isCorrect, (25,25), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                if (left_arm_visible):
                    
                    #checking if left arm is straight
                    if l_angle<180 and l_angle> 70:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    cv2.putText(image,"Left arm okay: " +isCorrect, (25,50), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                    #Checking if the angle between shoulder and left arm is correct
                    if l_alt_angle>160 and l_alt_angle<180:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    cv2.putText(image,"Left shoulder okay: "+isCorrect, (25,75), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                else:
                    cv2.putText(image,"Left arm is not visible properly ", (25,50), cv2.FONT_HERSHEY_TRIPLEX, 1, (252, 248, 3), 1)
                
                
                if (right_arm_visible):
                    
                    #checking if right arm is straight
                    if r_angle<180 and r_angle> 70:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0, 0,255)
                    cv2.putText(image,"Right arm okay: "+isCorrect, (25,100), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                    #Checking if the angle between shoulder and left arm is correct
                    if r_alt_angle>160 and r_alt_angle<180:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    cv2.putText(image,"right shoulder okay: "+isCorrect, (25,125), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                else:
                    cv2.putText(image,"Right arm is not visible properly ", (25,100), cv2.FONT_HERSHEY_TRIPLEX, 1, (252, 248, 3), 1)
                
            except:
                pass

            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(145,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(100,66,230), thickness=2, circle_radius=2) 
                                    )               
        out.write(image)   
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    out.release()
    cap.release()  
    cv2.destroyAllWindows()

In [ ]:
#chest press
#cap = cv2.VideoCapture("http://10.0.0.185:8080/video")
cap = cv2.VideoCapture("Chest_Press.mp4")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#Creating output file to save video 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_path = 'Chest_press_test.mp4'
fps_out = 25 # frame rate (frames per second), over 30 may not work
out = cv2.VideoWriter(video_path, fourcc, fps_out, (width, height))

running = False
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.6) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        k = cv2.waitKey(33) # THIS GRABS THE KEY VALUE

        # Show the wireframe
        if(k == 116):
            running = True
        # Turn off the wireframe.
        if(k == 121):
            running = False
        
        
        if(running):
            
            
            try:
                landmarks = results.pose_landmarks.landmark
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                ls_visibility = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                rs_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
                
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                le_visibility = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                re_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
                
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                rw_visibility = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                lw_visibility = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
                
                left_knee_angle = calculate_angle(left_ankle, left_knee, left_hip)
                right_knee_angle = calculate_angle(right_ankle, right_knee, right_hip)
                
                right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
                left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
                
                left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                
                l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                r_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                
                l_alt_angle = calculate_angle(right_shoulder, left_shoulder, left_elbow)
                r_alt_angle = calculate_angle(left_shoulder, right_shoulder, right_elbow)
                
                left_arm_visible = ls_visibility>0.8 and le_visibility>0.8 and lw_visibility>0.8
                right_arm_visible = rs_visibility>0.8 and re_visibility>0.8 and rw_visibility>0.8
                
                
                if left_knee_angle>170 and right_knee_angle>170 and left_hip_angle>150 and right_hip_angle>150:
                    isCorrect = "True"
                    color = (0,255,0)
                else:
                    isCorrect = "False"
                    color = (0,0,255)
                cv2.putText(image,"Lower body position Okay: "+isCorrect, (25,25), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                
                
                #caculating the height difference between shoulders
                shoulder_Y_diff =  abs(int(left_shoulder[1]*frame_h) -int(right_shoulder[1]*frame_h))
                
                
                #checking if shoulders are in parallel position
                if shoulder_Y_diff>10:
                    color = (0,0,255)
                    if int(left_shoulder[1]*frame_h)>int(right_shoulder[1]*frame_h):
                        upper_shoulder = "Left shoulder"
                    else:
                        upper_shoulder = "Right Shoulder"
                    cv2.putText(image,"Shoulders are not parralel, " +upper_shoulder+" needs to be lower", (25,50), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                else:
                    color = (0,255,0)
                    cv2.putText(image,"Shoulders are in parralel position.", (25,50), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                   
                if (left_arm_visible):
                    
                    #checking if left arm is in correct form
                    if l_angle<180 and l_angle> 60:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0,0,255)
                    cv2.putText(image,"Left arm okay: " +isCorrect, (25,75), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                else:
                    cv2.putText(image,"Left arm is not visible properly ", (25,75), cv2.FONT_HERSHEY_TRIPLEX, 1, (252, 248, 3), 1)
                    
       
                if (right_arm_visible):
                    
                    #checking if right arm is in correct form
                    if r_angle<180 and r_angle> 60:
                        isCorrect = "True"
                        color = (0,255,0)
                    else:
                        isCorrect = "False"
                        color = (0, 0,255)
                    cv2.putText(image,"Right arm okay: "+isCorrect, (25,100), cv2.FONT_HERSHEY_TRIPLEX, 1, color, 1)
                    
                    
                else:
                    cv2.putText(image,"Right arm is not visible properly ", (25,100), cv2.FONT_HERSHEY_TRIPLEX, 1, (252, 248, 3), 1)
                
            except:
                pass

            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(145,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(100,66,230), thickness=2, circle_radius=2) 
                                    )               
        out.write(image)    
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    out.release()
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#chest press
#cap = cv2.VideoCapture("http://10.0.0.185:8080/video")
import cv2 as cv
image = cv.imread("TRX (2).jpg")
running = False
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.6) as pose:
        
    # Recolor image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
      
    # Make detection
    results = pose.process(image)
    
    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(0,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(0,66,230), thickness=2, circle_radius=2) 
                                    )               
       
    cv.imwrite("t.jpg",image)
    cv2.imshow('Mediapipe Feed', image)
    